In [1]:
import os
import sys
import string
import time
import datetime
import pandas as pd
import numpy as np
from collections import OrderedDict

sys.path.insert(1, '../../../MayomicsVC/testing')
from integration_test import *

In [2]:
wdl_base_dir = '../../../MayomicsVC/src/wdl'
types_dict = get_task_types(wdl_directory=wdl_base_dir)
for k, v in types_dict.items():
    print('%30s: %s'%(k,v))

                          File: 123
                        String: 114
                         File?: 1
                       Boolean: 2
                   Array[File]: 2


In [44]:
def get_wdl_file_variables_dict(full_filename):
    """ Usage  config_orderd_dict = get_wdl_file_variables_dict(full_filename, task_types_list)
    Get the sorted list of variables and their types from the wdl files in a directory tree
    
    Args:
        wdl_directory:       (default to run directory if not valid directory name)
        
    Returns:
        config_ordered_dict: python OrderedDict of  variable_name: "type_name" 
    """
    skip_line_chars = ['#', '<', '>']
    base_path, fname = os.path.split(full_filename)
    types_dict = get_task_types(wdl_directory=base_path)
    for k, v in types_dict.items():
        print('%30s: %s'%(k,v))
    task_types_list = list(types_dict.keys())
    config_vars_dict = {}
    with open(full_filename, 'r') as fh:
        lines = fh.readlines()
        
    for line in lines:
        l = line.strip()
        if len(l) > 0 and not l[0] in skip_line_chars:
            line_words_list = l.split()
            first_word = line_words_list[0]
            if first_word in task_types_list:
                second_word = line_words_list[1]
                if not second_word in config_vars_dict.keys():
                    config_vars_dict[second_word] = '"' + first_word + '"'

                elif first_word != config_vars_dict[second_word][1:-1]:
                    bugger = config_vars_dict[second_word][:-1]
                    config_vars_dict[second_word] = bugger + ', ' + first_word  + '"'
                                
    config_od = OrderedDict()
    for k, v in sorted(config_vars_dict.items()):
        config_od[k] = v

    return config_od

def get_file_segment(full_filename, section_word):
    """ Usage: segment_text_list = get_file_segment(full_filename, section_word) """
    opener_char = '{'
    opener_count = 0
    closer_char = '}'
    closer_count = 0
    segment_text_list = []
    with open(full_filename, 'r') as fh:
        lines = fh.readlines()
        
    for line in lines:
        if len(line) > 0:
            l = line.strip()
            if opener_count > closer_count:
                segment_text_list.append(l)
                opener_count += l.count(opener_char)
                closer_count += l.count(closer_char)
                
            if section_word in l:
                segment_text_list.append(l)
                opener_count += l.count(opener_char)
                closer_count += l.count(closer_char)
                
    return segment_text_list

def get_typed_vars(text_list):
    """ Usage: vars_dict = get_typed_vars(text_list) """
    vars_dict = {}
    skippies = ['{']
    for line in text_list:
        l = line.strip().split()
        if len(l) ==  2:
            skipit = False
            for w in l:
                if skippies[0] in w:
                    skipit = True
            if skipit == False:
                vars_dict[l[1]] = l[0]
                
    typed_vars_od = OrderedDict()
    for k, v in sorted(vars_dict.items()):
        typed_vars_od[k] = v
        
    return typed_vars_od

def get_file_segment_variables(full_filename, section_word):
    """ Usage: vars_type_orderdict = get_file_segment_variables(full_filename, section_word) """
    return get_typed_vars(get_file_segment(full_file, section_word))
    
def get_import_files_dict(full_filename):
    """ Usage: files_dict = get_import_files_dict(full_filename) """
    import_string = 'import'
    files_dict = {}
    with open(full_filename, 'r') as fh:
        lines = fh.readlines()
        
    for line in lines:
        line_list = line.strip().split()
        if len(line_list) > 1 and line_list[0] == import_string:
            files_dict[line_list[-1]] = line_list[1].strip('"')
            
    return files_dict

In [45]:
wdl_base_dir = '../../../MayomicsVC/src/wdl'
# types_dict = ingrat.get_task_types(wdl_directory=wdl_base_dir)
wdl_file_inq = 'GermlineMasterWorkflow.wdl'
full_file = os.path.join(wdl_base_dir, wdl_file_inq)

files_dict = get_import_files_dict(full_filename = full_file)
for k, f in files_dict.items():
    print('%30s: %s'%(k,f))

                  CUTADAPTTRIM: MayomicsVC/src/wdl/Alignment/TestTasks/Runtrim_sequences.wdl
                     ALIGNMENT: MayomicsVC/src/wdl/Alignment/TestTasks/Runalignment.wdl
                      MERGEBAM: MayomicsVC/src/wdl/Alignment/Tasks/merge_aligned_bam.wdl
                         DEDUP: MayomicsVC/src/wdl/Alignment/Tasks/dedup.wdl
             DELIVER_Alignment: MayomicsVC/src/wdl/DeliveryOfAlignment/Tasks/deliver_alignment.wdl
                   REALIGNMENT: MayomicsVC/src/wdl/HaplotyperVC/Tasks/realignment.wdl
                          BQSR: MayomicsVC/src/wdl/HaplotyperVC/Tasks/bqsr.wdl
                    HAPLOTYPER: MayomicsVC/src/wdl/HaplotyperVC/Tasks/haplotyper.wdl
                          VQSR: MayomicsVC/src/wdl/HaplotyperVC/Tasks/vqsr.wdl
          DELIVER_HaplotyperVC: MayomicsVC/src/wdl/DeliveryOfHaplotyperVC/Tasks/deliver_HaplotyperVC.wdl


In [34]:
section_word = 'workflow'
wdl_base_dir = '../../../MayomicsVC/src/wdl'
# types_dict = ingrat.get_task_types(wdl_directory=wdl_base_dir)
wdl_file_inq = 'GermlineMasterWorkflow.wdl'
full_file = os.path.join(wdl_base_dir, wdl_file_inq)

vars_type_orderdict = get_file_segment_variables(full_file, section_word)
for v, vt in vars_type_orderdict.items():
    print('%30s: %s'%(v,vt))

                          Bqsr: Boolean
                MarkDuplicates: Boolean
              NormalInputReads: Array[Array[File]]
                      Trimming: Boolean
                          Vqsr: Boolean


In [4]:
wdl_base_dir = '../../../MayomicsVC/src/wdl'
# types_dict = ingrat.get_task_types(wdl_directory=wdl_base_dir)
wdl_file_inq = 'GermlineMasterWorkflow.wdl'
full_file = os.path.join(wdl_base_dir, wdl_file_inq)
# os.listdir(wdl_base_dir)
task_types_list = list(types_dict.keys())

if os.path.isfile(full_file):
    config_orderd_dict = get_wdl_file_variables_dict(full_file)
    for k, v in config_orderd_dict.items():
        print('%30s: %s'%(k,v))

                          File: 123
                        String: 114
                         File?: 1
                       Boolean: 2
                   Array[File]: 2
                          Bqsr: "Boolean"
        DeliverAlignOutputBais: "File"
        DeliverAlignOutputBams: "File"
          DeliverHaplotyperVcf: "File"
       DeliverHaplotyperVcfIdx: "File"
                MarkDuplicates: "Boolean"
                      Trimming: "Boolean"
                          Vqsr: "Boolean"
